# This Notebook has been superceded by the underCompMain.py program. 

### Run it from the command line as follows:
$python underCompMain.py --runs 5 --infile ../../data/csl/CSL_d5_transfus_yes.csv --target transfus_yes --sampling 1.0 --seed 1 --outdir underCompOut

### The following cell runs it in this Notebook
### (Note: it takes about 10 min. per run.)

In [9]:
import underCompMain
%run underCompMain --runs 2 --infile ../../data/csl/CSL_d5_transfus_yes.csv --target transfus_yes --sampling 1.0 --seed 1 --outdir underCompOut


Counter({1: 5170, 2: 5170})
Hyp_alpha
Hyp_clon
Hyp_othr
TTTwin
R_preg


/MFMDatasets/MFM_bopf/src/imbl/stat_mwb.py:17: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))


Counter({1: 5170, 2: 5170})
AB_doxy
Hyp_alpha
Hyp_othr
TTTwin
R_preg


/MFMDatasets/MFM_bopf/src/imbl/stat_mwb.py:17: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))


                   corr0     corr1  mean_corr
transfus_yes    0.999807  0.999807   0.999807
hemorrhage      0.977235  0.977613   0.977424
Sitenum         0.603643  0.602629   0.603136
Intratocolytix  0.479686  0.482502   0.481094
Inoxy_incrdose  0.478325  0.483212   0.480768
Education       0.435762  0.417416   0.426589
HospElectCS     0.401976  0.407649   0.404813
DMControl       0.395251  0.402858   0.399055
HosEpiNurse     0.378899  0.383222   0.381061
Hostype         0.344316  0.353286   0.348801
Admcontract     0.334410  0.339022   0.336716
AdmBishop       0.320335  0.322197   0.321266
GAmethod        0.302069  0.307698   0.304884
EBLoss          0.299992  0.306932   0.303462
Delfetalpos     0.276447  0.292444   0.284445
Analgesia       0.285110  0.281129   0.283120
Augment         0.277120  0.270369   0.273745
Admconsistency  0.266202  0.272321   0.269262
AdmSBP          0.269060  0.254446   0.261753
Admcervpos      0.256812  0.265383   0.261097
                rank0  rank1  mean

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

In [3]:
%%time
import util_mwb
#df = pd.read_csv('../../data/csl/CSL_data5_mod.csv', index_col=0)
#df = pd.read_csv('../../data/csl/CSL_d5_ty-mnm.csv', index_col=0)
df = pd.read_csv('../../data/csl/CSL_d5_transfus_yes_2020-07.csv', index_col=0)
df

CPU times: user 4.47 s, sys: 778 ms, total: 5.25 s
Wall time: 5.28 s


,Sitenum,Momdeath,MomICU,transfus_yes,AB_any,AB_ceph,AB_clind,AB_doxy,AB_eryth,AB_flagyl,...,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_Gravidity,high_MomLOS,high_height
MomID,,,,,,,,,,,,,,,,,,,,,
41-00002,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,2,3,2.0
41-00003,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,3,2,2.0
41-00004,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,4,1,3.0
41-00005,41,1,1,2,1,1,1,1,1,1,...,1,0,1,1,1,0,0,3,2,2.0
41-00006,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,1,2,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-07247,52,1,1,1,1,1,1,1,1,1,...,1,0,1,1,3,0,3,1,2,2.0
52-07248,52,1,1,1,1,1,1,1,1,1,...,1,1,1,1,3,0,0,2,2,3.0
52-07249,52,1,2,2,1,1,1,1,1,1,...,1,1,1,1,3,0,0,3,3,2.0


In [4]:
df.shape

(185413, 239)

In [5]:
X = df.drop('transfus_yes', axis=1, inplace=False)
y = df['transfus_yes'].values
print(X.shape)
print(y.shape)

(185413, 238)
(185413,)


In [6]:
# Verify no columns with null values
for col in X:
    if X[col].isnull().values.any():
        print(col)

In [11]:
print(f'y = {y}')
X

y = [1 1 1 ... 2 1 1]


,Sitenum,Momdeath,MomICU,AB_any,AB_ceph,AB_clind,AB_doxy,AB_eryth,AB_flagyl,AB_gent,...,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_Gravidity,high_MomLOS,high_height
MomID,,,,,,,,,,,,,,,,,,,,,
41-00002,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,2,3,2.0
41-00003,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,3,2,2.0
41-00004,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,4,1,3.0
41-00005,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,3,2,2.0
41-00006,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,1,2,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52-07247,52,1,1,1,1,1,1,1,1,1,...,1,0,1,1,3,0,3,1,2,2.0
52-07248,52,1,1,1,1,1,1,1,1,1,...,1,1,1,1,3,0,0,2,2,3.0
52-07249,52,1,2,1,1,1,1,1,1,1,...,1,1,1,1,3,0,0,3,3,2.0


In [11]:
%%time
rand_und = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_res, y_res = rand_und.fit_resample(X, y)

CPU times: user 882 ms, sys: 0 ns, total: 882 ms
Wall time: 885 ms


In [12]:
y_res_s = pd.Series(y_res)
xind = X.columns
X_res_df = pd.DataFrame(X_res, columns=xind)

In [13]:
print(Counter(y))
print(Counter(y_res))

Counter({1: 180243, 2: 5170})
Counter({1: 5170, 2: 5170})


In [14]:
X_res_df.head()

,Sitenum,Momdeath,MomICU,AB_any,AB_ceph,AB_clind,AB_doxy,AB_eryth,AB_flagyl,AB_gent,...,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_Gravidity,high_MomLOS,high_height
0,45,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,3,3,2.0
1,49,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,1,4,3,2.0
2,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,1,2,2.0
3,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,2,2,3.0
4,43,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,3,2,2,2.0


In [15]:
X_res_df.iloc[0,:]

Sitenum           45.0
Momdeath           1.0
MomICU             1.0
AB_any             1.0
AB_ceph            1.0
                  ... 
high_BMI           0.0
high_Age           0.0
high_Gravidity     3.0
high_MomLOS        3.0
high_height        2.0
Name: 0, Length: 238, dtype: float64

In [16]:
for col in X_res_df:
    X_res_df[col] = X_res_df[col].astype(int)

In [17]:
X_res_df

,Sitenum,Momdeath,MomICU,AB_any,AB_ceph,AB_clind,AB_doxy,AB_eryth,AB_flagyl,AB_gent,...,ich_new1,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_Gravidity,high_MomLOS,high_height
0,45,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,3,3,2
1,49,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,1,4,3,2
2,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,1,2,2
3,41,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,0,2,2,3
4,43,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,0,3,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10335,52,1,1,1,1,1,1,1,1,1,...,1,0,1,1,5,0,0,2,2,2
10336,52,1,1,1,1,1,1,1,1,1,...,1,1,1,1,3,0,0,1,3,2
10337,52,1,1,1,1,1,1,1,1,1,...,1,1,1,1,3,0,0,3,2,2
10338,52,1,1,1,1,1,1,1,1,1,...,1,0,1,1,5,0,0,4,2,2


In [18]:
pd.options.display.max_rows = None
counts = X_res_df.apply(lambda x: Counter(x))
counts

Sitenum                {45: 483, 49: 893, 41: 5005, 43: 670, 48: 469,...
Momdeath                                                {1: 10334, 2: 6}
MomICU                                                {1: 10199, 2: 141}
AB_any                                                {1: 8856, 2: 1484}
AB_ceph                                                {1: 9988, 2: 352}
AB_clind                                              {1: 10151, 2: 189}
AB_doxy                                                       {1: 10340}
AB_eryth                                               {1: 10273, 2: 67}
AB_flagyl                                               {1: 10333, 2: 7}
AB_gent                                               {1: 10098, 2: 242}
AB_peni                                                {1: 9554, 2: 786}
AB_vanc                                                {1: 10325, 2: 15}
abruptio9                                             {0: 10081, 1: 259}
abruption                                          

In [19]:
pd.options.display.max_rows = 20

In [30]:
import os
y_res_s.name = 'transfus_yes'

#Xfile = '../../data/csl/und_Rand_1-ty_100_Xresult.csv'
#yfile = '../../data/csl/und_Rand_1-ty_100_yresult.csv'
Xfile = '../../data/csl/und_Rand_199-ty_100_Xresult.csv'
yfile = '../../data/csl/und_Rand_199-ty_100_yresult.csv'
if not os.path.exists(Xfile):
    X_res_df.to_csv(Xfile, header=True)
if not os.path.exists(yfile):
    y_res_s.to_csv(yfile, header=True)

In [32]:
all_df = pd.DataFrame(X_res_df)
all_df.insert(0,'transfus_yes',y_res_s)
all_df

,transfus_yes,Sitenum,Momdeath,MomICU,AB_any,AB_ceph,AB_clind,AB_doxy,AB_eryth,AB_flagyl,...,iufd9,myocardio_new1,R_preg,Outcome1,high_BMI,high_Age,high_EBLoss,high_Gravidity,high_MomLOS,high_height
0,1,41,1,1,2,1,1,1,1,1,...,0,1,1,1,0,0,1,2,2,2
1,1,49,1,1,1,1,1,1,1,1,...,0,1,1,1,1,0,0,2,2,2
2,1,50,1,1,1,1,1,1,1,1,...,0,1,1,1,0,0,1,1,3,2
3,1,51,1,1,1,1,1,1,1,1,...,0,1,1,1,0,1,0,4,2,3
4,1,47,1,1,1,1,1,1,1,1,...,0,1,1,1,0,0,0,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10335,2,52,1,1,1,1,1,1,1,1,...,0,1,1,5,0,0,2,2,2,2
10336,2,52,1,1,1,1,1,1,1,1,...,1,1,1,3,0,0,0,1,3,2
10337,2,52,1,1,1,1,1,1,1,1,...,1,1,1,3,0,0,0,3,2,2
10338,2,52,1,1,1,1,1,1,1,1,...,0,1,1,5,0,0,1,4,2,2


In [33]:
#allfile = '../../data/csl/und_Rand_0-ty_100.csv'
#allfile = '../../data/csl/und_Rand_1-ty_100.csv'
allfile = '../../data/csl/und_Rand_199-ty_100.csv'
if not os.path.exists(allfile):
    all_df.to_csv(allfile, header=True)